# Standard Transformer Example

This notebook demonstrates how to deploy a PyFunc model and a standard transformer. The pyfunc model simply return the request from the transformer. While the transformer itself retrieve the feature from Feast server.

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 1. Initialize Merlin

### 1.1 Set Merlin Server

In [ ]:
import merlin
print(merlin.__version__)

MERLIN_URL = "merlin-api-url"

merlin.set_url(MERLIN_URL)

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project-name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP UI.

In [ ]:
PROJECT_NAME = "sample"

merlin.set_project(PROJECT_NAME)

### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in Merlin is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by Merlin are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [ ]:
from merlin.model import ModelType

MODEL_NAME = "pyfunc-standard-transfo"

merlin.set_model(MODEL_NAME, ModelType.PYFUNC)

## 2. Create PyFunc Model

To create a PyFunc model you'll have to extend `merlin.PyFuncModel` class and implement its `initialize` and `infer` method.

`initialize` will be called once during model initialization. The argument to `initialize` is a dictionary containing a key value pair of artifact name and its URL. The artifact's keys are the same value as received by `log_pyfunc_model`.

`infer` method is the prediction method that needs to be implemented. It accept a dictionary type argument which represent incoming request body. `infer` should return a dictionary object which correspond to response body of prediction result.

In following example we are creating PyFunc model called `StandardModel`. 
This model will simply echo-ing back the request body to its sender and print `feast_features` that will be populated by standard transformer into stdout.

In [ ]:
import pandas as pd
import logging
from merlin.model import PyFuncModel

class StandardModel(PyFuncModel):
    def initialize(self, artifacts):
        pass
        
    def infer(self, request, **kwargs):
        if "feast_features" in request:
            logging.info(pd.DataFrame(**request["feast_features"]["merchant_id"]))
        return request

Now, let's test it locally.

In [ ]:
m = StandardModel()
m.initialize({})
m.infer(
    {
    "merchants": [
        {
            "id" : "M111"
        },
        {
            "id" : "M222"
        },
        {
            "id" : "M333"
        }
    ]
    }
)

Test whether it could accept feature enrichment from standard transformer. Note that `feast_features` json field will be populated by standard transformer, and the format follow pandas.DataFrame with `split` orientation.

In [ ]:
m.infer(
    {
        "merchants": [
        {
            "id" : "M111"
        },
        {
            "id" : "M222"
        },
        {
            "id" : "M333"
        }
    ], 
        # feast_features will be populated by standard transformer
        "feast_features": {
          "merchant_id" : {
            "columns": [
                "merchant_id",
                "merchant_discovery:delivery_time_estimate"
            ],
            "data": [["M111", 10],["M222", 20],["M333", 30], ]
          }
        }
    }
)

## 3. Deploy Model

To deploy the model, we will have to create an iteration of the model (by create a `model_version`), upload the serialized model to MLP, and then deploy.

### 3.1 Create Model Version and Upload

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_pyfunc_model(model_instance=EnsembleModel(), 
                conda_env="env.yaml", 
                artifacts={"xgb_model": model_1_path, "sklearn_model": model_2_path})
v.finish()
```

To upload PyFunc model you have to provide following arguments:
1. `model_instance` is the instance of PyFunc model, the model has to extend `merlin.PyFuncModel`
2. `conda_env` is path to conda environment yaml file. The environment yaml file must contain all dependency required by the PyFunc model.
3. (Optional) `artifacts` is additional artifact that you want to include in the model
4. (Optional) `code_path` is a list of directory containing python code that will be loaded during model initialization, this is required when `model_instance` depend on local python package

In [ ]:
with merlin.new_model_version() as v:    
    merlin.log_pyfunc_model(model_instance=StandardModel(),
                            conda_env="standard_transformer_env.yaml",
                            artifacts={})

### 3.2 Deploy Model and Transformer

To deploy a model and its transformer, you must pass a `transformer` object to `deploy()` function. Each of deployed model version will have its own generated url.

In [ ]:
!cat "standard_transformer_config.yaml"

In [ ]:
from merlin.resource_request import ResourceRequest
from merlin.transformer import StandardTransformer
from merlin.logger import Logger, LoggerConfig, LoggerMode

# Create a transformer object and its resources requests
transformer_config_path = "standard_transformer_config.yaml"
transformer = StandardTransformer(config_file=transformer_config_path,
                                  enabled=True)

log = Logger(model=LoggerConfig(enabled=True,  mode=LoggerMode.ALL))
endpoint = merlin.deploy(v, transformer=transformer, logger=log)

### 3.3 Send Test Request

In [ ]:
import requests

resp = requests.post(endpoint.url, json={
    "merchants": [
        {
            "id" : "M111"
        },
        {
            "id" : "M222"
        },
        {
            "id" : "M333"
        }
    ]
    }
)
resp.text

## 3. Clean Up

## 3.1 Delete Deployment

In [ ]:
merlin.undeploy(v)